# Security Measures for Supply Chain Management System

This notebook details the implementation of crucial security measures to protect the automated supply chain management system. We will focus on AWS Identity and Access Management (IAM), AWS Key Management Service (KMS), and data encryption protocols to ensure robust security.


## Step 1: Create IAM Roles and Policies

We start by creating IAM roles and policies that strictly limit permissions to only the necessary AWS resources.


In [ ]:
import boto3

iam = boto3.client('iam')

# Create an IAM policy
policy_document = """
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "dynamodb:*",
        "lambda:*",
        "s3:*",
        "iot:*",
        "kinesis:*"
      ],
      "Resource": "*"
    }
  ]
}
"""
policy_response = iam.create_policy(
  PolicyName='MySupplyChainPolicy',
  PolicyDocument=policy_document
)

# Create an IAM role
trust_policy = """
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {"Service": "lambda.amazonaws.com"},
      "Action": "sts:AssumeRole"
    }
  ]
}
"""
role_response = iam.create_role(
  RoleName='MySupplyChainRole',
  AssumeRolePolicyDocument=trust_policy
)

# Attach the policy to the role
attach_response = iam.attach_role_policy(
  RoleName='MySupplyChainRole',
  PolicyArn=policy_response['Policy']['Arn']
)

policy_response, role_response, attach_response


## Step 2: Use AWS KMS for Key Management

Next, we create a customer-managed key in AWS KMS to manage the encryption of our data.


In [ ]:
kms = boto3.client('kms')

# Create a customer-managed key
key_response = kms.create_key(
  Description='Key for encrypting supply chain data'
)

# Use the key to encrypt data
encryption_example = kms.encrypt(
  KeyId=key_response['KeyMetadata']['KeyId'],
  Plaintext='Sensitive supply chain data'
)

key_response, encryption_example


## Step 3: Enable Encryption at Rest and in Transit

It's crucial to ensure that data is encrypted both at rest and in transit to prevent unauthorized access.


In [ ]:
s3 = boto3.client('s3')
dynamodb = boto3.client('dynamodb')

# Enable server-side encryption for an S3 bucket using the KMS key
s3.put_bucket_encryption(
  Bucket='your-bucket-name',
  ServerSideEncryptionConfiguration={
    'Rules': [{
      'ApplyServerSideEncryptionByDefault': {
        'SSEAlgorithm': 'aws:kms',
        'KMSMasterKeyID': key_response['KeyMetadata']['KeyId']
      }
    }]
  }
)

# Enable encryption at rest for DynamoDB
dynamodb_response = dynamodb.update_table(
  TableName='YourDynamoDBTableName',
  SSESpecification={
    'Enabled': True,
    'SSEType': 'KMS',
    'KMSMasterKeyId': key_response['KeyMetadata']['KeyId']
  }
)

dynamodb_response


# Conclusion

By implementing these security measures, we have significantly strengthened the defenses of our automated supply chain management system. IAM roles and policies provide granular access control, while AWS KMS ensures that our data is securely encrypted, protecting it from unauthorized access.
